# Exploratory Data Analysis of Engineered Features

Observations:
* Moderate to weak visual correlations between engineered features and targets
* Score of first assessment looks to have a sigificant collelation with estimated final score, final outcome
* Individual distributions show a lot of 0 values, unsurprising otherwise

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
sns.reset_defaults
sns.set_style(style='whitegrid')
sns.set_context(context='poster')
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
font = {'size'   : 16}
plt.rc('font', **font)
plt.ion()
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True
plt.rcParams["figure.figsize"] = (20.0, 10.0)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)# reg_df = pd.read_csv('../data/raw/studentRegistration.csv')
# reg_df = pd.read_csv('../data/raw/studentRegistration.csv')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, confusion_matrix, recall_score, roc_auc_score, roc_curve, recall_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
def scale_subset(df, columns):
    '''
    Use sklearn StandardScalar to scale only numeric columns.

    Parameters:
    ----------
    input {dataframe, list}: dataframe containing mixed feature variable types, list of names of numeric feature columns
    output: {dataframe}: dataframe with numeric features scaled and categorical features unchanged

    '''
    scalar = StandardScaler()
    numeric = df[columns]
    categorical = df.drop(columns, axis = 1)
    scalar.fit(numeric)
    num_scaled = pd.DataFrame(scalar.transform(numeric))
    num_scaled.rename(columns = dict(zip(num_scaled.columns, numeric_cols)), inplace = True)
    return pd.concat([num_scaled, categorical], axis = 1)

In [2]:
df = pd.read_csv('../data/processed/first_half/transformed_data_with_features.csv')

In [6]:
X_train = pd.read_csv('../data/processed/first_half/X_train.csv')

In [7]:
X_train.columns

Index(['num_of_prev_attempts', 'studied_credits', 'module_presentation_length',
       'sum_click_dataplus', 'sum_click_dualpane', 'sum_click_externalquiz',
       'sum_click_forumng', 'sum_click_glossary', 'sum_click_homepage',
       'sum_click_htmlactivity', 'sum_click_oucollaborate',
       'sum_click_oucontent', 'sum_click_ouelluminate', 'sum_click_ouwiki',
       'sum_click_page', 'sum_click_questionnaire', 'sum_click_quiz',
       'sum_click_repeatactivity', 'sum_click_resource',
       'sum_click_sharedsubpage', 'sum_click_subpage', 'sum_click_url',
       'sum_days_vle_accessed', 'max_clicks_one_day',
       'first_date_vle_accessed', 'avg_score', 'avg_days_sub_early',
       'days_early_first_assessment', 'score_first_assessment',
       'code_module_BBB', 'code_module_CCC', 'code_module_DDD',
       'code_module_EEE', 'code_module_FFF', 'code_module_GGG',
       'code_module_nan', 'code_presentation_2013J', 'code_presentation_2014B',
       'code_presentation_2014J', 'code_p

Looking for data leakage

In [ ]:
df['date_unregistration'].hist(bins=80)

In [ ]:
df[df['date_unregistration'] >= 127].shape

In [ ]:
df['days_early_first_assessment'].isna().sum()

In [ ]:
df = df.fillna(value=0)

In [ ]:
df.shape

In [ ]:
df = df[df['date_unregistration'] >= 0]

## Single vs Double Modules

In [ ]:
plt.hist(df['estimated_final_score'])
plt.show()

In [ ]:
df[df['estimated_final_score'] > 100]['code_module_DDD'].sum()

Of students with a score higher that 100, 1696 were in module DDD

In [ ]:
df[df['estimated_final_score'] > 100].shape

In [ ]:
subdf = df[(df['code_module_DDD'] == 1) & (df['estimated_final_score'] > 100)][['code_presentation_2013J',
       'code_presentation_2014B', 'code_presentation_2014J', 'code_presentation_nan']]

In [ ]:
subdf.columns

In [ ]:
for col in subdf.columns:
    print(sum(subdf[col]))

**Conclusion: Double modules are code DDD, presenation 2013J, 2014B**

Changing the values:

In [ ]:
index_list = []
for index, row in df.iterrows():
    if row['code_module_DDD'] == 1:
        index_list.append(index)
#         row['estimated_final_score'] = row['estimated_final_score'] * 0.5

In [ ]:
len(index_list)

In [ ]:
plt.hist(df['estimated_final_score'])

In [ ]:
df.columns

In [ ]:
X = df.drop(['id_student', 'date_unregistration', 'module_not_completed', 'score_above_100', 'final_result', 'score_above_100'], axis =1)
y = df['score_above_100']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [ ]:
numeric_cols = ['num_of_prev_attempts', 'studied_credits', 'clicks_per_day', 'pct_days_vle_accessed','max_clicks_one_day','first_date_vle_accessed', 'avg_score', 'avg_days_sub_early','days_early_first_assessment', 'score_first_assessment']

In [ ]:
#     X_train.fillna(value = 0, inplace = True)
#     y_train.fillna(value = 0, inplace = True)
# #     X_train = scale_subset(X_train, numeric_cols)
#     X_test.fillna(value = 0, inplace = True)
#     y_test.fillna(value = 0, inplace = True)
# #     X_test = scale_subset(X_test, numeric_cols)

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
pred = rf.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
print(confusion_matrix(y_test, pred))

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df[(df['date_unregistration'].isnull())].shape

In [ ]:
df.fillna(value=0, inplace=True)

In [ ]:
df.corr()

In [ ]:
df.columns

## Engineered Feature Analysis

### Bivariate Plots Against Targets: final_result_num, estimated_final_score

In [ ]:
sns.jointplot(x='clicks_per_day', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='clicks_per_day', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='pct_days_vle_accessed', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='pct_days_vle_accessed', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='studied_credits', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='studied_credits', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
df.columns

In [ ]:
sns.jointplot(x='max_clicks_one_day', y='final_result_num', data=df, size=12, alpha=0.5)

In [ ]:
sns.jointplot(x='max_clicks_one_day', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='first_date_vle_accessed', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='first_date_vle_accessed', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_days_sub_early', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_days_sub_early', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
df.columns

In [ ]:
sns.jointplot(x='days_early_first_assessment', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='days_early_first_assessment', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='score_first_assessment', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='score_first_assessment', y='estimated_final_score', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_score', y='final_result_num', data=df, size=12, alpha=0.2)

In [ ]:
sns.jointplot(x='avg_score', y='estimated_final_score', data=df, size=12, alpha=0.1)

### Univariate Plots of Engineered Features

In [ ]:
df.columns

In [ ]:
f = list(df.columns[[1,2,4,6,7,8,9,10,11,12,13,14,15,16]])

In [ ]:
for feat in f:
    plt.figure(figsize=(12,12))
    sns.distplot(df[feat], bins=50, kde=False)